In [4]:
import pandas as pd

In [ ]:
class Ball(dict):
    def __init__(self):
        self['score'] = 0
        self['bat'] = 0
        self['wicket'] = None
        self['w_nb'] = True

In [ ]:
class Innings(Ball):
    
    def __init__(self):
        self.bat_order = []
        
    def balls(self):
        return [b for b in self.values() if isinstance(b, Ball)]
    
    def runs(self):
        return [b['score'] for b in self.balls()]
    
    def partnerships(self):
        return []
    
    def bat_score(self, batsman):        
        return sum(b['bat'] for b in self.balls() if (b['batsman'] == batsman))

    def bat_balls_faced(self, batsman):        
        return sum(1 for b in self.balls() if (b['batsman'] == batsman) & (b['w_nb']))
    
    def bat_scorecard(self):
        score_board = []
        for batsman in self.bat_order:
            runs = self.bat_score(batsman)
            balls = self.bat_balls_faced(batsman)
            score_board.append((batsman, runs, balls))
        
        sb = pd.DataFrame(data=score_board, columns=['Batsmen', 'Runs', 'Balls Faced'])

        total = ('Total', sum(self.runs()), len(self.balls()))
        extra = ('Extras', total[1] - sb['Runs'].sum(), total[2] - sb['Balls Faced'].sum())
        score_board.append(extra)
        score_board.append(total)

        return pd.DataFrame(data=score_board, columns=['Batsmen', 'Runs', 'Balls Faced'])       
 

In [ ]:
class Match(dict):    
    def __init__(self):
        self.team = {}
        self.team[1] = {}
        self.team[2] = {}
        self.team[1]['players'] = []
        self.team[1]['players'] = []
        
    def winner(self):
        pass       

In [68]:
game = Match()
f = open('IPL/335982.yaml', 'r')

for line in f.readlines():
    tabs = [a.strip('\n') for a in line.split('  ')]
    len_tabs = len(tabs)
    try:
        var, out = [a.strip() for a in tabs[-1].split(':')]
    except:
        var = None
        out = [a.strip() for a in tabs[-1].split('- ')]
   
    if len_tabs == 2:
        game[var] = out
        if (var == '- 1st innings'): 
            innings = game[1] = Innings()
        if (var == '- 2nd innings'):
            innings = game[2] = Innings()
            
    if len_tabs == 5:
        current_ball = innings[var] = Ball()

    if len_tabs == 7:
        if var == 'wicket':
            current_ball['wicket'] = True
        else:
            current_ball[var] = out
        if var == 'batsman':
            if out not in innings.bat_order:
                innings.bat_order.append(out)       
        
    if len_tabs == 8:
        if var == 'total':
            current_ball['score'] = int(out)
        if var == 'batsman':
            current_ball['bat'] = int(out)
        if (var == 'wides') | (var == 'no-ball'):
            current_ball['w_nb'] = False
        
            

In [69]:
game[1].bat_score('BB McCullum')

158

In [72]:
game[1].bat_scorecard()

,Batsmen,Runs,Balls Faced
0,SC Ganguly,10,12
1,BB McCullum,158,73
2,RT Ponting,20,20
3,DJ Hussey,12,12
4,Mohammad Hafeez,5,3
5,Extras,17,4
6,Total,222,124


In [35]:
sb

NameError: name 'sb' is not defined